# 칼로리 개정

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 데이터셋 로드
food = pd.read_csv('./foodData4_24.csv')

# 결측값 처리(Keywords열의 데이터 중 nan값 공백으로 처리)
# 현재 food 데이터에 일부 깨진데이터(?)가 있어서 결측값 처리해야 정상작동
food['Keywords'] = food['Keywords'].fillna('')

# TF-IDF(Term Frequency-Inverse Document Frequency)
# 텍스트 데이터의 통계적인 가중치를 계산, 처리해주는 기능(자연어 처리)

# TfidfVectorizer : 문서를 벡터 표현으로 바꿔주는 기능
# stop_words='english' => 영어의 일반적인 불용어 삭제(and, is, the, this 등 단어와 상관없는 문자)
tfidf = TfidfVectorizer(stop_words='english')

# food의 Keywords를 단어로 구별
# 단어로 구별된 Keywords를 TF-IDF 행렬로 생성
tfidf_matrix = tfidf.fit_transform(food['Keywords'])

# 사용자 정보 입력
user_info = {
    'Birth date': '26.02.1991',
    'Height': 171,
    'Weight': 74,
    'Name': 'Emmie',
    'Email': 'ecleator2@booking.com',
    'Gender': 'Female'
}

# BMI 계산
height_m = user_info['Height'] / 100
weight_kg = user_info['Weight']
bmi = weight_kg / (height_m ** 2)

# 추정된 칼로리 요구량 계산
if user_info['Gender'] == 'Male':
    bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_m * 100) - (5.677 * 33)  # 테스트를 위해 임의의 나이 사용
else:
    bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_m * 100) - (4.330 * 33)

calories_threshold = bmr * 1.2  # 활동 수준을 고려하여 칼로리 요구량을 조정

#----------------------------------------------------------------------------------------------------------여기까진 회원가입한 사용자의 정보를 읽어와서 BMI,BMR계산시킬 예정

# 필터링 및 추천 함수 정의
def get_filtered_recommendations(user_info, tfidf, tfidf_matrix, food_data):    # user_info => 'keywords', 'calories', 'nutrient_limit', 'excluded_ingredients', 'max_cooking_time'.
                                                                                #               를 사용하여 추천을 개인화 시킴.
                                                                                # tfidf, tfidf_matrix => 사용자 키워드와 음식데이터의 키워드를 벡터로 변환
    # 사용자 정보에서 필요한 데이터 추출
    user_keywords = user_info.get('keywords', [])
    user_calories = user_info.get('calories')
    nutrient_limit = user_info.get('nutrient_limit', {})
    excluded_ingredients = user_info.get('excluded_ingredients', [])    # 
    max_cooking_time = user_info.get('max_cooking_time')

    # 필수 인자 확인                                                     
    if tfidf_matrix is None:
        raise ValueError("TF-IDF 행렬이 제공되어야 합니다.")
    if food_data is None:
        raise ValueError("음식 데이터가 제공되어야 합니다.")
    if user_calories is None:
        raise ValueError("사용자의 칼로리가 제공되어야 합니다.")
    
    # 적어도 하나 이상의 필터링 옵션을 선택해야 함
    if not user_keywords and not nutrient_limit and not excluded_ingredients and max_cooking_time is None:
        raise ValueError("적어도 하나 이상의 필터링 옵션을 선택해야 합니다.")
    
    # 필요한 값이 없는 경우 기본값 설정
    if not user_keywords:
        user_keywords = []
    if not nutrient_limit:
        nutrient_limit = {}
    if not excluded_ingredients:
        excluded_ingredients = []
    if max_cooking_time is None:
        max_cooking_time = float('inf')
    
    # 사용자 입력에 따라 추천 음식 필터링
    user_tfidf_matrix = tfidf.transform([' '.join(user_keywords)])
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:]
    food_indices = [idx[0] for idx in sim_scores]
    recommended_food = food_data.iloc[food_indices][['Name', 'Calories', 'FatContent', 'CarbohydrateContent', 'ProteinContent', 'RecipeInstructions']]
    recommended_food = recommended_food[recommended_food['Calories'] <= user_calories]
    if nutrient_limit:
        for nutrient, limit in nutrient_limit.items():
            recommended_food = recommended_food[recommended_food[nutrient] <= limit]
    if excluded_ingredients:
        for ingredient in excluded_ingredients:
            recommended_food = recommended_food[~recommended_food['RecipeInstructions'].str.contains(ingredient, case=False)]
    if max_cooking_time:
        recommended_food = recommended_food[recommended_food['RecipeInstructions'].str.contains(f'(?i)\\b(?:time|clock|minutes|hour|min|second|sec|hr|h|s|duration)\\b\\s*{max_cooking_time}', regex=True, case=False)]
    return recommended_food
  # 사용자가 선택한 키워드를 TF-IDF 행렬 형식으로 변환.
  # 사용자 키워드의 TF-IDF행렬과 음식 데이터 TF-IDF행렬 사이의 코사인 유사도 계산 및 측정
  # 각 음식에 대한 유사도 점수를 열거후, 리스트로 변환
  # 유사도 점수를기준으로 내림차순으로 정렬
  # 가장 유사한 음식은 사용자가 이미 알고있으므로, 첫번째 음식을 제외하고 추천리스트를 만듦.
  # 추천 리스트에서 각 음식의 인덱스 추출
  # 추천된 음식의 데이터를 추출(이름, 열량, 영양정보, 레시피)
  # 추천된 음식 중 칼로리가 사용자가 설정한 칼로리 제한 이하인 음식만을 선택하게함.
  # 사용자가 영양소 제한을 설정한 경우, 해당 영양소의 제한에 맞는 음식만을 선택함.
  # 사용자가 제외하고자 하는 재료를 설정한 경우, 해당 재료를 포함한 음식은 제외.
  # 사용자가 설정한 최대 조리 시간 이내에 조리할 수 있는 음식만을 선택
  # 필터링된 음식 목록 반환

# 사용자 정보 입력
users = {
    'user1': { 
        'keywords': ['Dessert', 'Pork', 'Grains', 'Oven', 'Christmas', 'Brunch'],
        'calories': calories_threshold / 3,
        'nutrient_limit': {'FatContent': 20, 'ProteinContent': 30},
        'excluded_ingredients': ['Egg', 'Milk'],
        'max_cooking_time': 30
    },
    'user2': {
        'keywords': ['Dessert', 'Berries', 'Fruit', 'Low Protein', 'Brunch'],
        'calories': calories_threshold / 2,
        'nutrient_limit': {'CarbohydrateContent': 50, 'ProteinContent': 20},
        'excluded_ingredients': ['Peanut', 'Shellfish'],
        'max_cooking_time': 45
    },
    'user3': {
        'keywords': ['Egg Free', 'Fruit', 'Oven', 'Brunch', 'Berries'],
        'calories': 150,
        'nutrient_limit': {'ProteinContent': 10, 'CarbohydrateContent': 30},
        'excluded_ingredients': ['Wheat', 'Gluten'],
        'max_cooking_time': 20
    }
}

# 각 사용자에 대한 음식 추천 실행
for user, info in users.items():
    print(f"{user}의 필터된 음식 추천:")
    recommendations = get_filtered_recommendations(info, tfidf, tfidf_matrix, food)
    print(f"{user}의 BMI: {bmi}")
    print(f"{user}의 BMR: {bmr}")
    print(recommendations)
    print()


user1의 필터된 음식 추천:
user1의 BMI: 25.3069320474676
user1의 BMR: 1518.739
                                                    Name  Calories  \
115882                          Orange Glazed Spiral Ham      70.2   
88915                                  Caramel Chai Bars     162.5   
132764                            Sourdough Pumpernickel      82.2   
154296     Sourdough Bread in Custom-Programmable Zo ABM     194.1   
93274                                 Sancho Panza Spuds     203.2   
29219               Lemon and Red Onion Roasted Potatoes     205.3   
5279                       Easy Low-Fat Turkey Meatballs     264.5   
54978                        Grilled Seasoned Pork Roast     292.1   
71919                          Chicken and Vegi Stir Fry     343.4   
114686              My Own Dough Recipe for Breadmakers.     390.5   
114795                        Baked Beans With Pineapple     254.6   
116992                          Delicious Collard Greens     300.2   
119018                